In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm, trange

In [3]:
datapath = Path('data')

In [4]:
df_train = pd.read_csv(datapath / 'train.csv')
df_train.head()
y_train = df_train['target']

In [5]:
df_test = pd.read_csv(datapath / 'test.csv')
df_test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [6]:
def dedup_test(df_test):
    unique_samples = []
    unique_count = np.zeros_like(df_test)
    for feature in range(df_test.shape[1]):
        _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
        unique_count[index_[count_ == 1], feature] += 1

    # Samples which have unique values are real the others are fake
    real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
#     synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]
    return df_test[real_samples_indexes]

In [7]:
deduped_test = dedup_test(df_test.drop(columns=['ID_code']).values)

In [8]:
df_test = pd.DataFrame(deduped_test, columns=df_test.drop(columns=['ID_code']).columns)
df_test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
1,17.3035,-2.4212,13.3989,8.3998,11.0777,9.6449,5.9596,17.8477,-4.8068,7.4643,...,4.4676,4.4214,0.9303,1.4994,15.2648,-1.7931,6.5316,10.4855,23.4631,0.7283
2,10.6137,-2.1898,8.9090,3.8014,13.8602,-5.9802,5.5515,15.4716,-0.1714,7.6178,...,13.1683,4.0625,-0.1537,7.9787,18.4518,0.1000,-7.8212,9.2355,15.0721,-7.3475
3,14.8595,-4.5378,13.6483,5.6480,9.9144,1.5190,5.0358,13.4524,-2.5419,9.4450,...,2.6735,5.8526,4.8517,2.5020,22.8224,-0.9325,8.6849,10.2848,17.4932,6.0800
4,14.1732,-5.1490,9.7591,3.7316,10.3700,-21.9202,7.7130,18.8749,0.4680,7.8453,...,0.8640,5.9058,1.3140,4.8961,20.1087,1.1051,7.7184,9.3406,21.1746,-2.0098


In [12]:
def get_categorical_for_var(var_i):
    var_s = f'var_{var_i}'
    categorical = np.zeros(df_train.shape[0], dtype=int)
    unq_val, unq_idx, unq_inv, unq_cnt = np.unique(df_train[var_s], return_index=True, 
                                                   return_inverse=True, return_counts=True)
#     print(unq_val.shape, unq_idx.shape, unq_inv.shape, unq_cnt.shape)
    for i in trange(df_train.shape[0]):
        val = df_train[var_s][i]
        cnt = unq_cnt[unq_inv[i]]
        targets_ones = None
        if cnt == 1:
            cnt_in_test = len(df_test[ df_test[var_s] == val ])
            if cnt_in_test == 0:
                category = 5
            else:
                category = 4
        else:
            targets_ones = df_train[ df_train[var_s] == val ]['target'].sum()
            if cnt == targets_ones:
                category = 1
            else:
                if targets_ones == 0:
                    category = 2
                else:
                    category = 3
        categorical[i] = category
#         print(i, cnt, targets_ones, category)
#         if i > 2000:
#             break
    return categorical

In [13]:
%time categorical_0 = get_categorical_for_var(0)

100%|██████████| 200000/200000 [03:28<00:00, 957.63it/s]

CPU times: user 3min 28s, sys: 1.14 s, total: 3min 29s
Wall time: 3min 28s


In [14]:
len(np.where( categorical_0 == 1)[0] ) + \
len(np.where( categorical_0 == 2)[0] ) + \
len(np.where( categorical_0 == 3)[0] ) + \
len(np.where( categorical_0 == 4)[0] ) + \
len(np.where( categorical_0 == 5)[0] )

200000

In [15]:
len(np.where( categorical_0 == 1)[0] ), \
len(np.where( categorical_0 == 2)[0] ), \
len(np.where( categorical_0 == 3)[0] ), \
len(np.where( categorical_0 == 4)[0] ), \
len(np.where( categorical_0 == 5)[0] )

(630, 113844, 45171, 17291, 23064)

In [16]:
var_s = 'var_0'

In [17]:
df_train_0 = df_train[df_train['target'] == 0]
df_train_1 = df_train[df_train['target'] == 1]

# Categoría 1: repetidos con target == 1

In [18]:
unq_val_1, unq_idx_1, unq_inv_1, unq_cnt_1 = np.unique(df_train_1[var_s], return_index=True, 
                                                   return_inverse=True, return_counts=True)


In [19]:
(unq_cnt_1[unq_inv_1] > 1).shape

(20098,)

In [20]:
categories1_mat_1 = np.zeros(len(df_train_1))
categories1_mat_1[ unq_cnt_1[unq_inv_1] > 1 ] = 1

In [21]:
categories1_mat_1[0:100]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.])

# Categoría 2: repetidos con target == 0

In [22]:
unq_val_0, unq_idx_0, unq_inv_0, unq_cnt_0 = np.unique(df_train_0[var_s], return_index=True, 
                                                   return_inverse=True, return_counts=True)

In [23]:
(unq_cnt_0[unq_inv_0] > 1).shape

(179902,)

In [24]:
categories2_mat_0 = np.zeros(len(df_train_0))
categories2_mat_0[ unq_cnt_0[unq_inv_0] > 1 ] = 2

In [25]:
categories2_mat_0.shape

(179902,)

# Categoria 3: repetidos entre target 0 y 1

In [26]:
idx_0 = np.in1d(df_train_0[var_s].values, df_train_1[var_s].values)
idx_1 = np.in1d(df_train_1[var_s].values, df_train_0[var_s].values)

In [27]:
len(idx_0) + len(idx_1)

200000

In [28]:
categories3_mat_0 = np.zeros(len(df_train_0))
categories3_mat_1 = np.zeros(len(df_train_1))
categories3_mat_0[ idx_0 ] = 3
categories3_mat_1[ idx_1 ] = 3
categories3_mat = np.append(categories3_mat_0, categories3_mat_1)

# Categoría 4: únicos en todo el train

In [29]:
df_total = np.append(df_train_0[var_s], df_train_1[var_s])

In [30]:
unq_val, unq_idx, unq_inv, unq_cnt = np.unique(df_total, return_index=True, 
                                                   return_inverse=True, return_counts=True)

In [31]:
categories4_mat = np.zeros(len(df_total))
categories4_mat[ unq_cnt[unq_inv] == 1 ] = 4

# Categoría 5: únicos train + test


In [32]:
df_train_test = np.append(df_total, df_test[var_s])

In [33]:
df_train_test.shape

(300000,)

In [34]:
unq_val_tt, unq_idx_tt, unq_inv_tt, unq_cnt_tt = np.unique(df_train_test, return_index=True, 
                                                   return_inverse=True, return_counts=True)

In [35]:
categories5_mat = np.zeros(len(df_train_test))
categories5_mat[ unq_cnt_tt[unq_inv_tt] == 1 ] = 5

In [36]:
categories5_mat = categories5_mat[0:len(df_total)]

# Merge de categorías

In [40]:
category_mat = np.append(categories2_mat_0, categories1_mat_1)

category_mat[ categories3_mat == 3 ] = 3

category_mat[ categories4_mat == 4 ] = 4

category_mat[ categories5_mat == 5 ] = 5

In [41]:
categorical_0[0:100]

array([2, 2, 2, 2, 2, 2, 3, 3, 4, 2, 2, 2, 2, 5, 2, 2, 5, 2, 2, 2, 2, 2,
       3, 4, 2, 3, 5, 2, 2, 3, 2, 2, 2, 2, 2, 5, 2, 2, 5, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 2, 4, 4, 4, 2, 2, 2, 2, 3, 4, 3, 2, 4, 2, 3, 5, 2, 3,
       2, 2, 4, 2, 2, 5, 3, 2, 5, 5, 2, 4, 2, 2, 2, 3, 5, 2, 3, 2, 2, 2,
       3, 2, 3, 3, 2, 2, 2, 2, 2, 2, 5, 3])

In [42]:
category_mat[0:100]

array([2., 2., 2., 2., 2., 2., 3., 3., 4., 2., 2., 2., 2., 2., 2., 5., 2.,
       2., 2., 2., 2., 3., 4., 2., 3., 5., 2., 2., 2., 2., 2., 2., 2., 5.,
       2., 2., 5., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2., 4., 4., 4., 2.,
       2., 2., 2., 3., 4., 3., 2., 4., 2., 3., 2., 2., 2., 4., 2., 2., 2.,
       5., 5., 2., 4., 2., 2., 2., 3., 5., 2., 2., 2., 2., 3., 2., 3., 3.,
       2., 2., 2., 2., 2., 2., 5., 2., 4., 2., 2., 2., 5., 3., 3.])

In [43]:
len(np.where( category_mat == 1)[0] ), \
len(np.where( category_mat == 2)[0] ), \
len(np.where( category_mat == 3)[0] ), \
len(np.where( category_mat == 4)[0] ), \
len(np.where( category_mat == 5)[0] )

(630, 113844, 45171, 17291, 23064)

In [44]:
len(np.where( categorical_0 == 1)[0] ), \
len(np.where( categorical_0 == 2)[0] ), \
len(np.where( categorical_0 == 3)[0] ), \
len(np.where( categorical_0 == 4)[0] ), \
len(np.where( categorical_0 == 5)[0] )

(630, 113844, 45171, 17291, 23064)